<a href="https://colab.research.google.com/github/2025-01-sookmyung-opensource/final_project/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필요한 패키지 설치
!pip install -q streamlit pyngrok ultralytics gdown opencv-python pillow

In [ ]:
# 2. pyngrok 임포트 및 인증 토큰 설정
from pyngrok import ngrok

# ngrok 인증 토큰 마스킹
ngrok.set_auth_token("2Np")

# 3. 기존에 열려있는 ngrok 세션 종료 (중복 방지)
!pkill -f ngrok

In [ ]:
# 모델 파일 다운로드
import gdown

# 1차 YOLOv8 탐지 모델 다운로드
gdown.download(id='1sl1dXyWuJiXjhHeacomWwfppsM57VJ85', output='best.pt', quiet=False)

# 2차 MobileNetV3 양 분류 모델 다운로드
gdown.download(id='16Veaxcko0DjfWoSaPqtfe5FVqXxZ25dB', output='quantity_model.pt', quiet=False)

In [ ]:
# 🖼️ app.py 저장
app_code = r"""
import streamlit as st
from PIL import Image
import numpy as np
import torch
import torchvision.transforms as transforms
import cv2
from ultralytics import YOLO
import torchvision.models as models
import torch.nn as nn

# ───────────────────────── 설정 ─────────────────────────
st.set_page_config(page_title='🍱 식단 이미지 분석기', layout='centered')
st.title('🍱 식단 이미지 분석기')
st.markdown('YOLOv8로 음식을 탐지하고, MobileNetV3로 **양 등급(Q1~Q5)** 을 예측합니다.')

# 등급 라벨
q_labels = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']

# ───────── 모델 로드 ─────────
@st.cache_resource
def load_models():
    # 1차 YOLOv8
    yolo = YOLO('best.pt')

    # 2차 MobileNetV3
    mobilenet = models.mobilenet_v3_small(weights=None)
    mobilenet.classifier[3] = nn.Linear(mobilenet.classifier[3].in_features, 5)
    mobilenet.load_state_dict(torch.load('quantity_model.pt', map_location='cpu'))
    mobilenet.eval()

    return yolo, mobilenet

yolo_model, quantity_model = load_models()

# 이미지 전처리 함수 (MobileNet용)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ───────── 업로드 처리 ─────────
uploaded = st.file_uploader("이미지를 업로드하세요", ['jpg', 'jpeg', 'png'])

if uploaded:
    pil_img = Image.open(uploaded).convert('RGB')
    rgb = np.array(pil_img)
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

    st.image(rgb, caption="업로드한 이미지", use_column_width=True)

    # 1차 탐지 (YOLO)
    res = yolo_model.predict(bgr, verbose=False)[0]
    names = yolo_model.names
    cls_ids = res.boxes.cls.cpu().numpy().astype(int)
    boxes   = res.boxes.xyxy.cpu().numpy()

    st.subheader("🍽️ 인식된 음식 목록")
    if len(cls_ids) == 0:
        st.write("음식을 인식하지 못했습니다.")
    else:
        for i, cid in enumerate(cls_ids):
            st.write(f"- {names[cid]}")

    st.image(res.plot(), caption="YOLO 인식 결과", use_column_width=True)

    st.subheader("📏 양 등급 분석 (Q1~Q5)")
    for i, (cid, box) in enumerate(zip(cls_ids, boxes), start=1):
        x1, y1, x2, y2 = map(int, box)
        crop_rgb = rgb[y1:y2, x1:x2]
        if crop_rgb.size == 0:
            continue

        # Crop → PIL → Transform
        pil_crop = Image.fromarray(crop_rgb)
        input_tensor = transform(pil_crop).unsqueeze(0)  # (1, 3, 224, 224)

        with torch.no_grad():
            output = quantity_model(input_tensor)
            _, predicted = torch.max(output, 1)
            grade = q_labels[predicted.item()]

        st.image(crop_rgb, caption=f"{i}. {names[cid]} - 양 등급: {grade}", width=300)
"""

with open("app.py", "w") as f:
    f.write(app_code)

In [ ]:
# 실행 및 공유
!nohup streamlit run app.py > /dev/null 2>&1 &
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f'앱이 실행 중입니다: {public_url}')